In [2]:
import json
import os 
import requests 
import pandas as pd 

/Users/alexsocarras/tech_impact/career-onestop/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


1. Create a demo copy of the survey in SurveyMonkey for development and testing. 

2. Test the SurveyMonkey API

- [x] Retrieve question/answers key from SurveyMonkey survey 
- [x] Combine with COS question/answer key to form a complete translation map 
- [x] Retrieve list of responses to SM survey 
- [x] Use map to translate survey responses to COS request objects  

3. Create Flask Web App based off this process.

---

#### 1. **Create a demo survey**

I created a demo survey in SurveyMonkey by copy-pasting questions and answers into the survey creation box: 


In [ ]:
import os
import json
import pandas as pd 

# Create matching survey in SM: import through text import (limited to 10 questions for free account)
with open(os.path.join("data","get-skills-api","response.json"), "r") as file: # previously extracted with API Explorer
    survey_questions = json.load(file)['Skills']

skill_level_map = { 
  "DataPoint20": "Beginner",
  "DataPoint35": "Basic",
  "DataPoint50": "Skilled",
  "DataPoint65": "Advanced",
  "DataPoint80": "Expert"
}

for q in survey_questions:
  for k,v in skill_level_map.items():
    q[v] = q.pop(k)

# Test format for copy pasting into SM web console
with open("survey-question-port.txt", "w") as file:
  for q in survey_questions[:9]:
    file.write(f"{q['ElementName']}: {q['Question']}\n")
    file.write(f"1 - Beginner: {q['AnchorFirst']}\n")
    file.write("2\n")
    file.write(f"3 - Skilled: {q['AnchorThrid']}\n")
    file.write("4\n")
    file.write(f"5 - Expert: {q['AnchorLast']}\n")
    file.write("\n")

![create-survey](img/create-survey.png)

Kamran took this survey -- I am using his responses for testing purposes. 

#### 2. **Testing Survey Monkey API**


-  Retrieve question/answer key from SurveyMonkey survey 
- Retrieve list of responses to SM survey 

#### Private App/self log-in

We may not need to have multiple SurveyMonkey accounts accessing our SurveyMonkey app. The only account we would need in such case would be the account we used to make the survey. If so, we can simply use the access token generated when we registered the app whenever we make our API calls: 


In [17]:
import requests 
import yaml 
import json

BASE_URL = 'https://api.surveymonkey.com/v3'

# Avoid Oauth 2.0 setup and temporary long-term credential creation for now 
with open("api-key.yaml", "r") as file:
     access_token = yaml.full_load(file)['sm']['app']['access-token']

headers = {
    'Authorization': f'Bearer {access_token}'
}

response = requests.get(f'{BASE_URL}/surveys', headers=headers)
response = response.json()

In [18]:
response

{'data': [{'id': '409346397',
   'title': 'Career Onestop Port',
   'nickname': '',
   'href': 'https://api.surveymonkey.com/v3/surveys/409346397'}],
 'per_page': 50,
 'page': 1,
 'total': 1,
 'links': {'self': 'https://api.surveymonkey.com/v3/surveys?per_page=50&page=1'}}

In [30]:
# Set survey id -- all calls will be made to /surveys/{SURVEY_ID}/ endpoint
SURVEY_ID = '409346397'
# SURVEY_ID = response['data'][0]['id']
SURVEY_ENDPOINT = f"{BASE_URL}/surveys/{SURVEY_ID}"

##### **Get SM Question/Answer Key**

In [368]:
# Get survey questions/answer option ids
# response = requests.get(SURVEY_ENDPOINT + "/details", headers=headers)
# survey_details = response.json()

## Save to avoid unnecessary calls to API
# import json
# with open("survey-details.json", "w") as file:
#     json.dump(survey_details, file)

with open("survey-details.json", "r") as file:
    survey_details = json.load(file)
# display(survey_details)

# Filtering for relevant information
smonkey_qa_map = []
for p in survey_details['pages']:
    for q in p['questions']:
        print(q)
        if 'answers' in q.keys(): 
            answer_choices = [{'text':d['text'], 'id':d['id']}
                 for d in q['answers']['choices']]
        else:
            answer_choices = None
        smonkey_qa_map.append({'id':q['id'],
            'heading':[h['heading'] for h in q['headings']],
            'answer_choices':answer_choices
        })
## Saving to reduce requests 
# with open("surveymonkey_smonkey_qa_map.json", "w") as file:
#     json.dump(smonkey_qa_map, file)

display(smonkey_qa_map)
len(smonkey_qa_map)

{'id': '151230949', 'position': 1, 'visible': True, 'family': 'open_ended', 'subtype': 'single', 'layout': {'left_spacing': 0, 'top_spacing': 0, 'right_spacing': 0, 'bottom_spacing': 0, 'width_format': None, 'width': None, 'col_width_format': None, 'col_width': None, 'position': 'new_row', 'num_chars': 100, 'num_lines': 3}, 'sorting': None, 'required': None, 'validation': {'type': 'email', 'text': 'Please enter a valid email address.', 'max': None, 'min': None, 'sum': None, 'sum_text': ''}, 'forced_ranking': False, 'headings': [{'heading': 'At what email address would you like to be contacted?'}], 'href': 'https://api.surveymonkey.com/v3/surveys/409346397/pages/45350810/questions/151230949'}
{'id': '150768414', 'position': 2, 'visible': True, 'family': 'single_choice', 'subtype': 'vertical', 'layout': None, 'sorting': None, 'required': None, 'validation': None, 'forced_ranking': False, 'headings': [{'heading': 'Administration and Management: How much do you know about business planning

[{'id': '151230949',
  'heading': ['At what email address would you like to be contacted?'],
  'answer_choices': None},
 {'id': '150768414',
  'heading': ['Administration and Management: How much do you know about business planning and leadership?'],
  'answer_choices': [{'text': '1 - Beginner: Complete a timesheet',
    'id': '1110937949'},
   {'text': '2', 'id': '1110937950'},
   {'text': '3 - Skilled: Monitor project progress to complete it on time',
    'id': '1110937951'},
   {'text': '4', 'id': '1110937952'},
   {'text': '5 - Expert: Manage a $10m company', 'id': '1110937953'}]},
 {'id': '150768415',
  'heading': ['Biology: How much do you know about plant, animal and cell functions?'],
  'answer_choices': [{'text': '1 - Beginner: Care for a pet',
    'id': '1110937954'},
   {'text': '2', 'id': '1110937955'},
   {'text': '3 - Skilled: Investigate effects of pollution on plants',
    'id': '1110937956'},
   {'text': '4', 'id': '1110937957'},
   {'text': '5 - Expert: Identify a new

10

##### **Combine with COS Question/Answer Key** 

(See `cs-api.ipynb` for code to make this json)

We only have a free tier SM account so we are limited to only 10 questions. The full skills-survey in CareerOneStop is 40 questions. 

We will therefore only combine what questions we have with the COS key for now.



In [334]:
with open("data/get-skills-api/response_translated.json", "r") as file:
    cos_skills_survey = json.load(file)

combined_map = [] # map for the skills survey questions
for n, question in enumerate(cos_skills_survey):
    d = {
        'question_text':{'cos':question['Question']},  # Add question text for human readability and double-checking match between questions 
        'question_number':{'cos':str(n + 1)}, 
        'question_id': {'cos':question['ElementId']},
        'answer_id':{
            'Beginner':{'cos':question['Beginner']},
            'Basic':{'cos':question['Basic']},
            'Skilled':{'cos':question['Skilled']},
            'Advanced':{'cos':question['Advanced']},
            'Expert':{'cos':question['Expert']},
        } 
    }
    combined_map.append(d)

## Version 1, more human readable 
skills_matcher_start_index = 1 # The Official DWDB SurveyMonkey survey includes contextual questions which aren't included in the CareerOneStop skills survey, as does our demo survey here
for n, q in enumerate(smonkey_qa_map[skills_matcher_start_index:]):

    # Match SM answer ids to COS answer ids 
    combined_map[n]['answer_id']['Beginner']['sm'] = q['answer_choices'][0]['id']
    combined_map[n]['answer_id']['Basic']['sm'] = q['answer_choices'][1]['id']
    combined_map[n]['answer_id']['Skilled']['sm'] = q['answer_choices'][2]['id']
    combined_map[n]['answer_id']['Advanced']['sm'] = q['answer_choices'][3]['id']
    combined_map[n]['answer_id']['Expert']['sm'] = q['answer_choices'][4]['id']
 

    # Add sm question id 
    combined_map[n]['question_id']['sm'] = q['id']

    # Add sm question number 
    combined_map[n]['question_number']['sm'] = str((n + 1) + skills_matcher_start_index) 

# display(combined_map[:1]) more human readable version of variable map
# with open("combined_map_v1.json", "w") as file:
#     json.dump(combined_map, file)

## Version 2 for actual translation 
# Change the 'Beginner', 'Basic', etc. skill levels to the SM answer ids
skill_levels = ('Beginner', 'Basic', 'Skilled', 'Advanced', 'Expert')
for n, d in enumerate(combined_map):
    for l in skill_levels:
        if 'sm' in d['answer_id'][l].keys():
            sm_answer_id = d['answer_id'][l]['sm']
            cos_answer_id = d['answer_id'][l]['cos']
            d['answer_id'].pop(l) 
            d['answer_id'][sm_answer_id] = cos_answer_id
# Cast to dictionary with SM question ids as keys
combined_map = {'skills-survey':{d['question_id']['sm']:d for d in combined_map if 'sm' in d['question_id'].keys()}}
# Add the non skills map questions -- TBD if we need to process these 
combined_map['non-skills-survey']  = [q for q in smonkey_qa_map[:skills_matcher_start_index]] 

display(combined_map)

with open("combined_map_v2.json", "w") as file:
    json.dump(combined_map, file)

{'skills-survey': {'150768414': {'question_text': {'cos': 'How much do you know about business planning and leadership?'},
   'question_number': {'cos': '1', 'sm': '2'},
   'question_id': {'cos': '2.C.1.a', 'sm': '150768414'},
   'answer_id': {'1110937949': 1.534,
    '1110937950': 2.4145,
    '1110937951': 3.295,
    '1110937952': 4.1755,
    '1110937953': 5.056}},
  '150768415': {'question_text': {'cos': 'How much do you know about plant, animal and cell functions?'},
   'question_number': {'cos': '2', 'sm': '3'},
   'question_id': {'cos': '2.C.4.d', 'sm': '150768415'},
   'answer_id': {'1110937954': 1.372,
    '1110937955': 2.401,
    '1110937956': 3.43,
    '1110937957': 4.459,
    '1110937958': 5.488}},
  '150768416': {'question_text': {'cos': 'How well can you coordinate moving your arms, legs, and torso together?'},
   'question_number': {'cos': '3', 'sm': '4'},
   'question_id': {'cos': '1.A.3.c.3', 'sm': '150768416'},
   'answer_id': {'1110937959': 1.176,
    '1110937960': 2.0

~~In our web app, we will run a check on the SurveyMonkey survey's 'last modified' field from the `/details` endpoint to see if we can use the cached copy of this main variable map, or else we will have to fetch and re-create a new map (TO-DO).~~

This is a moot point because to request the last modified value, you have to get the entire set of survey details anyways. 

##### **Get list of responses**
In production, this will be replaced with subscription to a SurveyMonkey webhoook.

In [354]:
## Get survey responses 
# response = requests.get(SURVEY_ENDPOINT + "/responses/bulk?per_page=100", headers=headers)
# survey_responses = response.json()
# survey_responses

## Save to avoid unnecessary calls to API
# import json
# with open("survey-responses.json", "w") as file:
#     json.dump(survey_responses, file)

with open("survey-responses.json", "r") as file:
    survey_responses = json.load(file)

# Filter for relevant information, and add COS data
new_responses = []
for resp in survey_responses['data']:
    resp_dict = {
        'resp_id':resp['id'],
        'collector_id':resp['collector_id'], 
        'questions':[]
    }
    n = 1
    for p in resp['pages']:
        for q in p['questions']:
            q_dict = {
                'question_number':{'sm':str(n+1)},
                'question_id':{'sm':q['id']},
                'question_type':None,
                'answer_id':{}
            }  
            n += 1 
            ## Adding COS translation data 
            q_dict['question_id']['cos'] = combined_map['skills-survey'][q['id']]['question_id']['cos']
            if q['id'] not in combined_map['skills-survey'].keys(): # if not a skills survey question
                q_dict['question_type'] = 'non-skills-survey' 
                q_dict['answer_id'] = q['answers'] # TBD if we need to process these answers 
            else: 
                q_dict['question_number']['cos'] = combined_map['skills-survey'][q['id']]['question_number']['cos']
                q_dict['question_type'] = 'skills-survey'
                sm_answer_id = q['answers'][0]['choice_id'] # skills survey questions only have one answer 
                q_dict['answer_id'] = {'sm':sm_answer_id, 
                                    'cos':combined_map['skills-survey'][q['id']]['answer_id'][sm_answer_id]}  
                
            resp_dict['questions'].append(q_dict)
    new_responses.append(resp_dict)

display(new_responses)   

[{'resp_id': '114409718452',
  'collector_id': '427785863',
  'questions': [{'question_number': {'sm': '2', 'cos': '1'},
    'question_id': {'sm': '150768414', 'cos': '2.C.1.a'},
    'question_type': 'skills-survey',
    'answer_id': {'sm': '1110937950', 'cos': 2.4145}},
   {'question_number': {'sm': '3', 'cos': '2'},
    'question_id': {'sm': '150768415', 'cos': '2.C.4.d'},
    'question_type': 'skills-survey',
    'answer_id': {'sm': '1110937954', 'cos': 1.372}},
   {'question_number': {'sm': '4', 'cos': '3'},
    'question_id': {'sm': '150768416', 'cos': '1.A.3.c.3'},
    'question_type': 'skills-survey',
    'answer_id': {'sm': '1110937962', 'cos': 3.822}},
   {'question_number': {'sm': '5', 'cos': '4'},
    'question_id': {'sm': '150768417', 'cos': '2.C.3.d'},
    'question_type': 'skills-survey',
    'answer_id': {'sm': '1110937967', 'cos': 4.03}},
   {'question_number': {'sm': '6', 'cos': '5'},
    'question_id': {'sm': '150768418', 'cos': '2.C.4.c'},
    'question_type': 'skill

In [3]:
with open("survey-responses.json", "r") as file:
    survey_responses = json.load(file)

survey_responses

{'data': [{'id': '114409718452',
   'recipient_id': '',
   'collection_mode': 'default',
   'response_status': 'completed',
   'custom_value': '',
   'first_name': '',
   'last_name': '',
   'email_address': '',
   'ip_address': '96.64.76.209',
   'logic_path': {},
   'metadata': {'contact': {}},
   'page_path': [],
   'collector_id': '427785863',
   'survey_id': '409346397',
   'custom_variables': {},
   'edit_url': 'https://www.surveymonkey.com/r/?sm=MKTi7zUaCT0NNV4xwYeK7ZznphgwMRWxwqKsI120yUFxpWGRWVD0l5Ev1zYBoZVK',
   'analyze_url': 'https://www.surveymonkey.com/analyze/browse/JrhdBA97A18icLNdv_2B26M4cs4Lx9WaWryCJ3TK_2F_2FzUk_3D?respondent_id=114409718452',
   'total_time': 41,
   'date_modified': '2023-09-11T16:21:35+00:00',
   'date_created': '2023-09-11T16:20:54+00:00',
   'href': 'https://api.surveymonkey.com/v3/surveys/409346397/responses/114409718452',
   'pages': [{'id': '45350810',
     'questions': [{'id': '150768414',
       'answers': [{'choice_id': '1110937950'}]},
     

In [370]:
survey_responses['data'][0].keys()

dict_keys(['id', 'recipient_id', 'collection_mode', 'response_status', 'custom_value', 'first_name', 'last_name', 'email_address', 'ip_address', 'logic_path', 'metadata', 'page_path', 'collector_id', 'survey_id', 'custom_variables', 'edit_url', 'analyze_url', 'total_time', 'date_modified', 'date_created', 'href', 'pages'])

Notice the response didn't include the non skills survey question from our demo survey (question #1, which asked for user email). Kamran did not fill this one out -- we will need to keep  in mind that uncompleted questions won't have their answer values included.

##### **Create request object for CareerOneStop Skills Matcher**


In [361]:
# Use our translated response to create a request object to the skills matcher 
with open(os.path.join("data", "skills-submit-api", "example-request.json"), "r") as file:
  example_request = json.load(file)

display(example_request['SKAValueList'][0])

new_cos_request = {'SKAValueList':[
  {'ElementId':q['question_id']['cos'], 'DataValue':q['answer_id']['cos']}
for resp in new_responses for q in resp['questions'] if q['question_type'] == 'skills-survey']}

new_cos_request

{'ElementId': '2.C.1.a', 'DataValue': '3.295'}

{'SKAValueList': [{'ElementId': '2.C.1.a', 'DataValue': 2.4145},
  {'ElementId': '2.C.4.d', 'DataValue': 1.372},
  {'ElementId': '1.A.3.c.3', 'DataValue': 3.822},
  {'ElementId': '2.C.3.d', 'DataValue': 4.03},
  {'ElementId': '2.C.4.c', 'DataValue': 2.331},
  {'ElementId': '2.C.1.b', 'DataValue': 2.3305},
  {'ElementId': '2.B.2.i', 'DataValue': 3.31},
  {'ElementId': '2.C.3.a', 'DataValue': 2.408},
  {'ElementId': '2.C.1.e', 'DataValue': 2.261}]}

See `cs-api.ipynb` for how to extend this type of request object with random answers for the rest of the skills survey questions (recall that we're limited to this number of questions by SM free tier) and how to send it in a GET request to the COS API.

---


#### Oauth 2.0 Setup

We won't need to use Oauth 2.0 unless we have multiple SurveyMonkey Accounts which will be logging into the app. We will come back to this section later on if we decide that is necessary. 

1. [] Register draft app (public for test, private requires enterprise license)
2. [] create website to host app
3. [] set up [oauth redirect](https://developer.surveymonkey.com/api/v3/#authentication) to app from host site 

**Step 1:** Send the user whose SurveyMonkey account you wish to access to a specially crafted OAuth link. The page presented to the user will identify your app, ask them to log into SurveyMonkey if they aren’t already, and ask them to authorize any required scopes

The OAuth link should be https://api.surveymonkey.com/oauth/authorize with urlencoded parameters: redirect_uri, client_id, response_type and state.

* "response_type"="code" (always, hardcoded value)
* "client_id"=unique client id you got when registering your app
* "redirect_uri"=URL encoded OAuth redirect URI you registered for your app
* "state" (recommended)=A value included in the request that will also be returned in the token response. It can be a string of any content that you wish. A randomly generated unique value is typically used for preventing cross-site request forgery attacks.*

e.g. `https://api.surveymonkey.com/oauth/authorize?response_type=code&redirect_uri=https%3A%2F%2Fapi.surveymonkey.com%2Fapi_console%2Foauth2callback&client_id=SurveyMonkeyApiConsole%2Fstate=uniquestring`

In [9]:
import yaml 
from urllib.parse import urlencode

oauth_link = "https://api.surveymonkey.com/oauth/authorize"

with open("api-key.yaml","r") as file:
    api_info = yaml.full_load(file)

RESPONSE_TYPE = "code"
REDIRECT_URI = "http://localhost:5000/auth/callback" # for local test
CLIENT_ID = api_info['sm']['app']['client-id']
# SECRET = api_info['sm']['app']['secret']
# ACCESS_TOKEN = api_info['sm']['app']['access_token']
print(RESPONSE_TYPE, REDIRECT_URI, CLIENT_ID)
params = {
    'response_type': RESPONSE_TYPE, 
    'redirect_uri': REDIRECT_URI, 
    'client_id': CLIENT_ID
}

full_oauth_link = f"{oauth_link}?{urlencode(params)}"
full_oauth_link

code http://127.0.0.1:5000/auth/callback C_44TfLERSi0kXy-tS2snQ


'https://api.surveymonkey.com/oauth/authorize?response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauth%2Fcallback&client_id=C_44TfLERSi0kXy-tS2snQ'